# 모델 및 train 데이터의 test_set

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/전력수요예측/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.models import load_model

# 모델 불러오기
loaded_model = load_model(path+'LSTM1_269.h5')
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 16)            1920      
                                                                 
 lstm_1 (LSTM)               (None, 24, 4)             336       
                                                                 
 lstm_2 (LSTM)               (None, 24, 1)             24        
                                                                 
 flatten (Flatten)           (None, 24)                0         
                                                                 
 dense (Dense)               (None, 8)                 200       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 2489 (9.72 KB)
Trainable params: 2489 (9.7

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# train 데이터 불러오기
X = np.load(path+'X_train_24.npy')
Y = np.load(path+'Y_train_24.npy')

print(X.shape)
print(Y.shape)

# test 할 데이터 추출

_, x_test, _, scaled_y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

del X,Y

(7585926, 24, 13)
(7585926, 1)


In [ ]:
# train 데이터의 test셋 예측
scaled_pred = loaded_model.predict(x_test)

# 스케일링 한 데이터 복구
y_max, y_min = 373.06, 48.9
y_test = scaled_y_test * (y_max-y_min) + y_min
pred = scaled_pred * (y_max-y_min) + y_min

47413/47413 [==============================] - 146s 3ms/step


In [ ]:
# 평가print('best iterations: {}'.format(xgb_reg.best_iteration))
print('SMAPE : {}'.format(mean_absolute_percentage_error(y_test, pred)*200))
print('MSE : {}'.format(mean_squared_error(y_test, pred)))
print('MAE : {}'.format(mean_absolute_error(y_test, pred)))
print('MAPE : {}'.format(mean_absolute_percentage_error(y_test, pred)))
print('R2   : {}'.format(r2_score(y_test, pred)))

SMAPE : 6.77483712362336
MSE : 22.723640458757146
MAE : 3.411201300504299
MAPE : 0.0338741856181168
R2   : 0.9640576178602794


# 제출용데이터 만들기

In [ ]:
#timestemps = 24 이므로
# 맨 앞의 23개의 데이터는 우선 100으로

test_file = pd.read_csv(path+'electric_test.csv')
test_file

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,2385,2023-01-01 01:00:00,1,303,7.8,61.5,6.7,0.0,4.2,6,1.0,NaN
1,2385,2023-01-01 02:00:00,2,303,7.9,60.6,7.6,0.0,4.0,6,1.0,NaN
2,2385,2023-01-01 03:00:00,3,303,8.2,61.9,8.7,0.0,4.1,6,1.0,NaN
3,2385,2023-01-01 04:00:00,4,303,8.4,60.9,9.2,0.0,4.3,6,1.0,NaN
4,2385,2023-01-01 05:00:00,5,303,8.5,60.9,9.5,0.0,4.3,6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2838234,12322,2023-12-31 19:00:00,19,901,5.7,72.3,3.6,0.0,5.7,6,1.0,NaN
2838235,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN
2838236,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN
2838237,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN


In [ ]:
test = np.load(path+'test_24.npy')

In [ ]:
test.shape

(2830787, 24, 13)

In [ ]:
# train 데이터의 test셋 예측
scaled_pred = loaded_model.predict(test)

# 스케일링 한 데이터 복구
y_max, y_min = 373.06, 48.9
pred = scaled_pred * (y_max-y_min) + y_min

88463/88463 [==============================] - 275s 3ms/step


In [ ]:
# timesteps로 작업하여 각 정점별 앞의 23개 데이터는 추측 불가
# 일단 100으로 채움
input_count = 0
for num in test_file['NUM'].unique():
    num_len = len(test_file.loc[test_file['NUM'] == num, 'elect'])

    test_file.loc[test_file['NUM'] == num, 'elect'] \
    = np.vstack([np.array([[100]]*23), pred[input_count:(input_count + num_len - 23)]])

    input_count = input_count + num_len - 23

print(input_count)

2830787


In [ ]:
test_file.iloc[:24,:]

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,2385,2023-01-01 01:00:00,1,303,7.8,61.5,6.7,0.0,4.2,6,1.0,100.000000
1,2385,2023-01-01 02:00:00,2,303,7.9,60.6,7.6,0.0,4.0,6,1.0,100.000000
2,2385,2023-01-01 03:00:00,3,303,8.2,61.9,8.7,0.0,4.1,6,1.0,100.000000
3,2385,2023-01-01 04:00:00,4,303,8.4,60.9,9.2,0.0,4.3,6,1.0,100.000000
4,2385,2023-01-01 05:00:00,5,303,8.5,60.9,9.5,0.0,4.3,6,1.0,100.000000
5,2385,2023-01-01 06:00:00,6,303,8.5,61.6,9.9,0.0,4.2,6,1.0,100.000000
6,2385,2023-01-01 07:00:00,7,303,8.5,60.1,9.1,0.0,4.4,6,1.0,100.000000
7,2385,2023-01-01 08:00:00,8,303,8.5,62.6,9.0,0.0,4.4,6,1.0,100.000000
8,2385,2023-01-01 09:00:00,9,303,8.3,65.2,7.4,0.0,4.6,6,1.0,100.000000
9,2385,2023-01-01 10:00:00,10,303,8.0,64.1,8.1,0.0,4.0,6,1.0,100.000000


In [ ]:
test_file.to_csv(path+'LSTM1_predict2.csv', index=False)